In [ ]:
import pandas as pd
import nibabel as nib

In [ ]:
patients = pd.read_csv('data/hemorrhage_diagnosis_raw_ct.csv')

h_start_idx = 2
h_end_idx = 7

some = []

ambiguous = []

#foo = True
for row_idx in range(patients.shape[0]):
    hemorrhage_row = patients.iloc[row_idx,h_start_idx:h_end_idx]
    hemorrhage_sum = hemorrhage_row.sum()
    if hemorrhage_sum > 1 or patients.iloc[row_idx, 0] == 51:
        ambiguous.append(patients.iloc[row_idx,:])
    if hemorrhage_sum > 0:
        some.append(patients.iloc[row_idx,:])


In [ ]:
df = pd.DataFrame(ambiguous)
df.iloc[:,h_start_idx:h_end_idx].sum(1)
some_df = pd.DataFrame(some)
n_total = some_df.iloc[:,h_start_idx:h_end_idx].shape[0]
n_ambiguous = df.iloc[:,h_start_idx:h_end_idx].shape[0]
n_unambiguous = n_total - n_ambiguous
print("Number of unambiguous slices: ", n_unambiguous )
print("Number of ambiguous slices: ", n_ambiguous )
print("Total slices with at least one lesion: ", n_total )

print("Patients with multiple hemorrhages:", df['PatientNumber'].unique().shape[0])
print("Nof Patients with any hemorrahge:", some_df['PatientNumber'].unique().shape[0])
print("Patients with any hemorrhage:", some_df['PatientNumber'].unique())
print(df['PatientNumber'].unique())

none = patients[~patients['PatientNumber'].isin(some_df['PatientNumber'])]['PatientNumber'].unique()
print(f"No lesions: {none}")
print(patients['PatientNumber'].unique().shape[0], none.shape[0], some_df['PatientNumber'].unique().shape[0])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import plot_utils
import torch
import networks.unet2D.utils as utils
color_map = torch.tensor([
            [0,0,0], # bg: black
            [0,1.0,0], # green, IVH
            [1.0,0,0], # red, IPH
            [0,0,1.0], # blue, SAH
            [0.0,0,0.5], # EDH, magenta?
            [0,0.5,0.9] # SDH, cyan?
    ])

def plot(patient_data):
    N = patient_data.shape[0]
    print(patient_data.shape)
    if N > 100:
        print("!!! TOO MUCH DATA TO SHOW; ABORTING !!!")
        return

    patient_numbers = patient_data['PatientNumber'].unique()
    counter = 0
    for patient_number in patient_numbers:
        print(patient_number)
        
            
        slices = patient_data[patient_data.PatientNumber == patient_number].SliceNumber - 1
        p_str = f'{patient_number}'
        if len(p_str) == 2:
            p_str = '0'+p_str

        img = nib.load(f'data/ct_scans/{p_str}.nii')
        img_data = img.get_fdata().clip(0, 150)

        mask_img = nib.load(f'data/multiclass_mask/{p_str}.nii')
        mask_img
        #mask_img = nib.load(f'data/masks/{p_str}.nii')
        mask_data = mask_img.get_fdata()#.clip(0, 150)

        for i in slices:
            
            mask_img = Image.fromarray(mask_data[:,:,i])
            src_img = Image.fromarray(img_data[:,:,i])
            
            mask = np.array(mask_img)
            src = np.array(src_img)
            if (mask.sum() > 0):
                #plt.imshow(mask, cmap='gray', origin='bottom')
                #plt.show()
                #plt.imshow(src, cmap='gray', origin='bottom')
                #plt.show()
                #plot_utils.plot_grid(mask,)
                ct = torch.tensor((img_data.clip(0,80) / 80)[:,:,i])
                mask_tensor = torch.tensor(mask_data[:,:,i])
                #plot_utils.plot_masked(ct, mask)
                if patient_number == 51 and i == 25:
                    print(mask.max(), mask_data[:,:,i].max())

                    print(mask_tensor.shape)
                    print(ct.unsqueeze(0).unsqueeze(0).shape)
                    mask_tensor = mask_tensor#.unsqueeze(0).unsqueeze(0)
                    print("class max", mask_tensor.min(), mask_tensor.max())
                    print(mask_tensor)
                    mask_tensor = mask_tensor.long()

                    max_idx = torch.argmax(mask_tensor)

                    mask_tensor = utils.labels_to_rgb_batched(mask_tensor.permute(0,2,3,1), color_map)
                    mask_tensor = mask_tensor.permute(0,3,1,2)
                    ct = ct.unsqueeze(0).unsqueeze(0).repeat(1,3,1,1)

                    print(mask_tensor.shape, ct.shape)
                    utils.save_overlay_grid(ct, mask_tensor, mask_tensor, 'ct_class_test.png')
            counter += 1
    print(counter)

plot(df)

In [ ]:
import plot_utils
import torch
from nibabel.testing import data_path
import os
import matplotlib.pyplot as plt
import numpy as np

patient_ids = patients.PatientNumber.unique()
emptied_slices = 0
for k, pid in enumerate(patient_ids):
    p_str = str(pid)
    if len(p_str) == 2:
        p_str = '0'+p_str

    print(p_str)

    modded_mask_save_path = f'data/multiclass_mask/{p_str}.nii'
    modded_img_save_path = f'data/ct_scans/{p_str}.nii'
    mask = nib.load(f'data/masks/{p_str}.nii')
    mask_data = mask.get_fdata()

    img = nib.load(f'data/ct_scans_backup/{p_str}.nii')
    img_data = img.get_fdata()
    modded_mask_data = np.zeros(mask_data.shape, dtype=np.uint8)
    #modded_img_data = np.zeros(img_data.shape)

    for i in range(mask_data.shape[2]):
        slice_annotation = patients[patients['PatientNumber'] == pid].iloc[i, h_start_idx:h_end_idx]
        n = slice_annotation.sum()
        #print(pid, i, n)
        if n == 1:
            # print(f"slice {i+1} has {n} bleeding type(s). Bleeding type(s): {slice_annotation}")
            class_id = np.array(slice_annotation).nonzero()[0][0] + 1

            #if pid == 51:
            #    print(class_id)
            modded_mask_data[:,:,i][mask_data[:,:,i] > 0] = class_id
        elif n > 1:
            # Setting the image to zero to prevent false negatives in training. Other possible approaches:
            # - randomize one of the classes for the mask (not ideal either)
            # - obtain proper unambiguous multi-class masks (ideal)
            img_data[:,:,i] = np.full((img_data.shape[0], img_data.shape[1]), fill_value=img_data.min())
            emptied_slices += 1

    #if pid == 51:
    #    print(modded_mask_data)

    modded_mask = nib.Nifti1Image(modded_mask_data, affine=mask.affine, header=mask.header)
    nib.save(modded_mask, modded_mask_save_path)

    modded_img = nib.Nifti1Image(img_data, affine=img.affine, header=img.header)
    nib.save(modded_img, modded_img_save_path)

print(f"Emptied {emptied_slices} slices.")

In [ ]:
import numpy as np

A = np.array([
    [
        [1,1,1],
        [1,1,1],
        [1,1,1]
    ],
    [
        [0,0,0],
        [1,1,0],
        [0,0,0]
    ]
])

modded_A = np.zeros(A.shape)
print(A.shape)
print(modded_A.shape)
modded_A[0][A[0] > 0] = 10
modded_A[1][A[1] > 0] = 20

modded_A

In [ ]:
# for k, pid in enumerate(patient_ids):
#     p_str = str(pid)
#     if len(p_str) == 2:
#         p_str = '0'+p_str
# 
#     print(p_str)
# 
#     mask = nib.load(f'data/multiclass_mask/{p_str}.nii')
#     mask_data = mask.get_fdata()
# 
#     for i in range(mask_data.shape[2]):
#         slice_annotation = patients[patients['PatientNumber'] == pid].iloc[i, h_start_idx:h_end_idx]
#         n = slice_annotation.sum()
#         if n > 1:
#             plt.hist(mask_data[:,:,i])
#             plt.show()


In [ ]:
import numpy as np
import os
from shutil import copyfile
from tqdm import tqdm
patients_with_some_hematoma = np.array(some_df['PatientNumber'].unique())

all_patient_ids = patients.PatientNumber.unique()
print("All patients: \n", all_patient_ids)

np.random.seed(123)
n_val = 3
n_test = 3

k = 5

validation_folds = []
test_folds = []
train_folds = []

test_fold = np.random.choice(patients_with_some_hematoma, n_test, replace=False) # Same test data for all folds for now

print("Creating folds...")
for i in range(k):
    post_test_selection_patients = patients_with_some_hematoma[np.isin(patients_with_some_hematoma, test_fold, invert=True)]
    validation_fold = np.random.choice(post_test_selection_patients, n_val, replace=False)
    #split_fold = np.split(fold, 2)
    #validation_fold, test_fold = split_fold[0], split_fold[1]
    already_reserved_data = np.concatenate((validation_fold, test_fold), axis=0)
    train_fold = all_patient_ids[np.isin(all_patient_ids, already_reserved_data, invert=True)]
    
    validation_folds.append(validation_fold)
    test_folds.append(test_fold)
    train_folds.append(train_fold)
    
print("Folds created!")

#[ print(fold) for fold in folds]
print("Validation folds: \n", validation_folds)
print("Test folds: \n", test_folds)
print("Train folds: \n", train_folds)

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def copy_files_in_fold(source_directory, target_directory, fold):
    for patient_number in tqdm(fold):#zip(validation_fold, test_fold, train_folds):
        p_str = f'{patient_number}'
        if len(p_str) == 2:
            p_str = '0'+p_str
        
        old_file = f'{source_directory}/{p_str}.nii'
        new_file = f'{target_directory}/{p_str}.nii'

        #print("")
        #print("old: ", old_file, " target: ", new_file)
        copyfile(old_file, new_file)

def copy_files_in_folds(base_directory, validation_folds, test_folds, train_folds):
    for i, (validation_fold, test_fold, train_fold) in enumerate(zip(validation_folds, test_folds, train_folds)):
        directory = f'{base_directory}/fold_{i}'

        create_dir(directory)

        val_directory = f'{directory}/validation'
        test_directory = f'{directory}/test'
        train_directory = f'{directory}/train'

        create_dir(val_directory)
        create_dir(test_directory)
        create_dir(train_directory)

        print("val test:")
        print(validation_fold, test_fold)
        copy_files_in_fold(base_directory, val_directory, validation_fold)
        copy_files_in_fold(base_directory, test_directory, test_fold)
        copy_files_in_fold(base_directory, train_directory, train_fold)

base_directory = 'data/multiclass_mask'
copy_files_in_folds(base_directory, validation_folds, test_folds, train_folds)

base_directory = 'data/ct_scans'
copy_files_in_folds(base_directory, validation_folds, test_folds, train_folds)

In [ ]:
copy_files_in_folds('data/ct_scans', validation_folds, test_folds, train_folds)

In [ ]:
import torch
from os import listdir
import numpy as np
import os
import shutil
from tqdm import tqdm

source_directory = 'E:\\kaggle\\rsna-intracranial-hemorrhage-detection\\nifti_down_sampled_train\\'
target_directory = 'E:\\kaggle\\rsna-intracranial-hemorrhage-detection\\nifti_down_sampled_test\\'
train_files = listdir(source_directory)
test_files = listdir(target_directory)
N_test = 5000

print(f'About to move {N_test} files. Source dir: {len(train_files)} file(s). Target dir: {len(test_files)} file(s).')
del test_files

def move_file(source_directory, target_directory, file_name):
    source_path = os.path.join(source_directory, file_name)
    target_path = os.path.join(target_directory, file_name)
    shutil.move(source_path, target_path)

def move_files(source_directory, target_directory, files, indices_to_move):
    for index_to_move in tqdm(indices_to_move):
        file_name = files[index_to_move]
        move_file(source_directory, target_directory, file_name)

    print(f'\nMoved {len(indices_to_move)} files.')

possible_indices = np.array(list(range(0,len(files))))
test_indices = np.random.choice(possible_indices, N_test, replace=False)
assert(np.unique(test_indices).shape == test_indices.shape)

move_files(
    source_directory=source_directory,
    target_directory=target_directory,
    files=train_files,
    indices_to_move=test_indices
)

In [ ]:
import torch
from os import listdir
import numpy as np
import os
import shutil
from tqdm import tqdm
import random

seed = 123
np.random.seed(seed)
random.seed(seed)

def write_list(root, file_names, N):
    with open(f'{root}/train_{N}.txt', 'w') as f:
        for f_n in file_names:
            f.write(f'{f_n}\n')

def select(file_names, N):
    return np.random.choice(file_names, N, replace=False)

def partition(root, file_names, Ns):
    file_names_left = file_names

    partitions = []
    for N in tqdm(Ns):
        file_names_left = select(file_names_left, N)
        write_list(root, file_names_left, N)
        partitions.append(file_names_left)

    partitions = list(reversed(partitions))
    for i, p in enumerate(partitions):
        all_larger = partitions[i+1:]
        assert(len(p) == len(np.unique(p)))
        for larger in all_larger:
            larger = np.array(larger)
            p = np.array(p)
            smaller_is_fully_included_in_larger = np.all(np.isin(p, larger))
            assert(smaller_is_fully_included_in_larger)

root = './data/experiment_data'
file_names = sorted(listdir('E:\\kaggle\\rsna-intracranial-hemorrhage-detection\\nifti_down_sampled\\'))
partitions = [10000, 8000, 5000, 4000, 2000, 1000, 500, 100]
partition(root, file_names, partitions)

In [ ]:
A = np.array(['foo', 'bar', 'barr'])
B = np.array(['foo', 'bar'])

np.alltrue(np.isin(B, A))

In [ ]:
import torch
from networks.dataset.ct import CtIchDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
# TODO: 0.8 negative sampling, update in real dataset?
# TODO: Don't use these directly in training, since it contains information from the test set
n_classes = 6
device = torch.device('cuda:0')
dataset = CtIchDataset(img_dir='../backup/data/all_scans/', mask_dir='../backup/data/all_masks/', N=-1, empty_mask_discard_probability=0.8, num_threads=1, resize=0.5, n_classes=6)
loader = DataLoader(dataset, batch_size=8, shuffle=False, pin_memory=True)


with torch.no_grad():
    counts = torch.zeros((n_classes,)).long().to(device)

    for i, (image, mask) in enumerate(tqdm(loader)):
        flat_mask = mask.view(-1).to(device)
        classes_present, class_counts = torch.unique(flat_mask, return_counts=True)
        counts[classes_present] = class_counts

    print(counts)
    print(counts)
    total = counts.sum().float()
    print(total)

    print(1 - counts.float() / total)

In [ ]:
print(counts)
total = counts.sum().float()
print(total)

1 - counts.float() / total

In [ ]:
A = torch.tensor([1,2,3,4,5])
A.tolist()

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(123)

patients = pd.read_csv('data/hemorrhage_diagnosis_raw_ct.csv')
h_start_idx = 2
h_end_idx = 7
N_negatives_val_test = 5

hemorrhages = patients.drop(['SliceNumber', 'Fracture_Yes_No'], axis='columns')
patients_grouped = patients.groupby(['PatientNumber']).agg({
    'Intraventricular': 'sum',
    'Intraparenchymal': 'sum',
    'Subarachnoid': 'sum',
    'Epidural': 'sum',
    'Subdural': 'sum',
    'No_Hemorrhage': 'sum',
    'SliceNumber': 'max'
    }).reset_index()

#some_lesion = hemorrhages.iloc[:,h_end_idx:h_start_idx].sum(axis=1)
#hemorrhages
patients_with_hemorrhage = patients_grouped[patients_grouped['No_Hemorrhage'] != patients_grouped['SliceNumber'] ]
patients_without_hemorrhage = patients_grouped[patients_grouped['No_Hemorrhage'] == patients_grouped['SliceNumber'] ]

ivh = patients_with_hemorrhage[patients_with_hemorrhage.Intraventricular > 1]
iph = patients_with_hemorrhage[patients_with_hemorrhage.Intraparenchymal > 1]
sah = patients_with_hemorrhage[patients_with_hemorrhage.Subarachnoid > 1]
edh = patients_with_hemorrhage[patients_with_hemorrhage.Epidural > 1]
sdh = patients_with_hemorrhage[patients_with_hemorrhage.Subdural > 1]


hems = ['Intraventricular', 'Intraparenchymal', 'Subarachnoid','Epidural','Subdural']

test_patients = np.full((5,), fill_value=-1)
val_patients = np.full((5,), fill_value=-1)
for i, hemorrhage in enumerate(hems):
    data_with_hemorrhage = np.array(patients_with_hemorrhage[patients_with_hemorrhage[hemorrhage] > 1].PatientNumber)
    # print(data_with_hemorrhage)

    taken = np.concatenate((test_patients, val_patients), axis=0)
    free_to_take = data_with_hemorrhage[np.isin(data_with_hemorrhage, taken, invert=True)]

    selected = np.random.choice(free_to_take, 2, replace=False)

    test = selected[0]
    val = selected[1]
    test_patients[i] = test
    val_patients[i] = val

patients_numbers_without_hemorrhage = np.array(patients_without_hemorrhage.PatientNumber)
test_negatives = np.random.choice(patients_numbers_without_hemorrhage, N_negatives_val_test, replace=False)
patients_numbers_without_hemorrhage = patients_numbers_without_hemorrhage[np.isin(patients_numbers_without_hemorrhage, test_negatives, invert=True)]
test_patients = np.concatenate((test_patients, test_negatives), axis=0)

val_negatives = np.random.choice(patients_numbers_without_hemorrhage, N_negatives_val_test, replace=False)
val_patients = np.concatenate((val_patients, val_negatives), axis=0)

all_patients = patients_grouped.PatientNumber.to_numpy()
taken = np.concatenate((test_patients, val_patients), axis=0)
train_patients = all_patients[np.isin(all_patients, taken, invert=True)]

print(test_patients, val_patients, train_patients)
print(f"in total: {test_patients.shape[0] + val_patients.shape[0] + train_patients.shape[0]}")

assert(np.intersect1d(val_patients, test_patients).size == 0)
assert(np.intersect1d(val_patients, train_patients).size == 0)
assert(np.intersect1d(test_patients, train_patients).size == 0)